In [29]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer


In [30]:
dataset = pd.read_pickle('final_dataset.pkl')

popData = pd.read_excel('COVID-19-geographic-disbtribution-worldwide.xlsx')
popData = popData.groupby(['countriesAndTerritories', 'countryterritoryCode', 'popData2019'], as_index=False)[['cases', 'deaths']].sum()
dataset['popData2019'] = popData.popData2019
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212 entries, 0 to 211
Data columns (total 15 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   countriesAndTerritories                212 non-null    object 
 1   countryterritoryCode                   212 non-null    object 
 2   percent_of_oct_cases_in_oct_and_sept   208 non-null    float64
 3   percent_of_oct_deaths_in_oct_and_sept  173 non-null    float64
 4   cases_since_sept_per100k               212 non-null    float64
 5   deaths_since_sept_per100k              212 non-null    float64
 6   avg_temp                               171 non-null    float64
 7   gdp_per_capita                         200 non-null    float64
 8   pop_density                            201 non-null    float64
 9   med_beds                               189 non-null    float64
 10  physicians                             194 non-null    float64
 11  pop_ab

Statystyki dla kolumn

In [31]:
dataset.describe()

,percent_of_oct_cases_in_oct_and_sept,percent_of_oct_deaths_in_oct_and_sept,cases_since_sept_per100k,deaths_since_sept_per100k,avg_temp,gdp_per_capita,pop_density,med_beds,physicians,pop_above_65,traffic_mortality,PM25,popData2019
count,208.000000,173.000000,212.000000,212.000000,171.000000,200.000000,201.000000,189.000000,194.000000,184.000000,168.000000,187.000000,2.120000e+02
mean,0.682832,0.661124,632.542397,7.635674,19.910217,17885.791503,326.578295,3.049534,1.863977,9.135931,17.065476,27.506042,3.618040e+07
std,0.207900,0.253320,934.702442,11.407682,7.283709,26650.498241,1488.559752,2.785252,1.666018,6.486272,9.304380,18.989965,1.408768e+08
min,0.000000,0.000000,0.000000,0.000000,-0.452322,126.924921,0.136492,0.100000,0.014000,1.156549,0.000000,5.861331,8.150000e+02
25%,0.549460,0.513710,16.113487,0.065787,13.509340,2121.690356,37.082592,1.200000,0.405825,3.409298,9.275000,13.866916,8.696470e+05
50%,0.694502,0.666667,178.221356,1.314206,22.823381,6916.301173,88.530570,2.300000,1.567450,6.967951,16.000000,22.196055,6.816581e+06
75%,0.852532,0.871287,914.870612,11.682601,25.586935,23141.109196,218.831818,4.000000,2.888825,14.970975,25.075000,35.644394,2.533154e+07
max,1.000000,1.000000,6232.852436,60.372024,30.771000,185829.017960,19196.000000,18.680000,8.421800,28.002049,35.900000,99.734374,1.433784e+09


Przypadki brakujące dla każdej kolumny

In [32]:
dataset.drop(columns=['percent_of_oct_cases_in_oct_and_sept', 'percent_of_oct_deaths_in_oct_and_sept']).isna().sum()

countriesAndTerritories       0
countryterritoryCode          0
cases_since_sept_per100k      0
deaths_since_sept_per100k     0
avg_temp                     41
gdp_per_capita               12
pop_density                  11
med_beds                     23
physicians                   18
pop_above_65                 28
traffic_mortality            44
PM25                         25
popData2019                   0
dtype: int64

In [33]:
imputer = KNNImputer(n_neighbors=10)
labels = dataset[['percent_of_oct_cases_in_oct_and_sept',
                'percent_of_oct_deaths_in_oct_and_sept',
                'countriesAndTerritories',
                'countryterritoryCode',
                'cases_since_sept_per100k',
                'deaths_since_sept_per100k']]
dataset_filled = dataset.drop(columns=['countriesAndTerritories',
                                       'countryterritoryCode',
                                       'percent_of_oct_cases_in_oct_and_sept',
                                       'percent_of_oct_deaths_in_oct_and_sept',
                                       'cases_since_sept_per100k',
                                       'deaths_since_sept_per100k'])

for column in dataset_filled.columns:
    column_mean = dataset_filled[column].mean()
    column_std = dataset_filled[column].std()
    dataset_filled[column] = (dataset_filled[column] - column_mean) / column_std

dataset_filled = pd.DataFrame(imputer.fit_transform(dataset_filled),
                  columns=dataset_filled.columns)

for column in dataset_filled:
    column_mean = dataset_filled[column].mean()
    column_std = dataset_filled[column].std()
    dataset_filled[column] = dataset_filled[column] * column_std + column_mean

In [34]:
dataset_filled = pd.concat([labels, dataset_filled], axis=1)
first_cols = dataset_filled[['countriesAndTerritories', 'countryterritoryCode']]
dataset_filled.drop(columns=['countriesAndTerritories', 'countryterritoryCode'], inplace=True)
dataset_filled.insert(0, 'countriesAndTerritories', first_cols['countriesAndTerritories'])
dataset_filled.insert(0, 'countryterritoryCode', first_cols['countryterritoryCode'])
dataset_filled.info()
dataset_filled.drop(columns=['popData2019'], inplace=True)
dataset_filled.to_pickle('final_filled_n_10_dataset.pkl')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212 entries, 0 to 211
Data columns (total 15 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   countryterritoryCode                   212 non-null    object 
 1   countriesAndTerritories                212 non-null    object 
 2   percent_of_oct_cases_in_oct_and_sept   208 non-null    float64
 3   percent_of_oct_deaths_in_oct_and_sept  173 non-null    float64
 4   cases_since_sept_per100k               212 non-null    float64
 5   deaths_since_sept_per100k              212 non-null    float64
 6   avg_temp                               212 non-null    float64
 7   gdp_per_capita                         212 non-null    float64
 8   pop_density                            212 non-null    float64
 9   med_beds                               212 non-null    float64
 10  physicians                             212 non-null    float64
 11  pop_ab

In [35]:
for key, data in dataset_filled.iteritems():
    if key not in ['countriesAndTerritories', 'countryterritoryCode']:
        print('Unusual values for key: ' + key)
        mean = data.mean()
        std = data.std()
        vals_series = data[(data <= mean - 3.0 * std) | (data >= mean + 3.0 * std)].index
        df = dataset_filled.loc[vals_series.values][['countriesAndTerritories', 'countryterritoryCode', key]]
        if df.empty:
            print('Brak wartości nietypowych')
        else:
            print(df)

Unusual values for key: percent_of_oct_cases_in_oct_and_sept
    countriesAndTerritories countryterritoryCode  \
28   British_Virgin_Islands                  VGB   
161   Saint_Kitts_and_Nevis                  KNA   

     percent_of_oct_cases_in_oct_and_sept  
28                               0.028571  
161                              0.000000  
Unusual values for key: percent_of_oct_deaths_in_oct_and_sept
Brak wartości nietypowych
Unusual values for key: cases_since_sept_per100k
    countriesAndTerritories countryterritoryCode  cases_since_sept_per100k
3                   Andorra                  AND               6232.852436
18                  Belgium                  BEL               3932.095962
51                  Czechia                  CZE               4091.616744
70         French_Polynesia                  PYF               4041.033353
130              Montenegro                  MNE               3598.143309
Unusual values for key: deaths_since_sept_per100k
    countries